## 1 task

In [146]:
import requests
import pickle
import time
import csv
import pandas as pd
import re

from IPython.display import display, HTML
from time import sleep
from bs4 import BeautifulSoup

pd.set_option('display.max_colwidth', -1)

URL = 'http://www.orsr.sk/vypis.asp'
NUM_COMPANIES = 50000

results = filter_results(get_pages())
df = pd.DataFrame([dict(info, Name=name) for result in results for name, info in result.items()])
columns = ['Business name', 'Name', 'Address']
df_top = df[df['Business name'].isin(df['Business name'].unique()[:NUM_COMPANIES])][columns]
df_russia = df_top.dropna()[df_top.dropna()['Address'].str.contains('Ruská|Rusko')]
df_top.to_csv('csv/1_task.csv', index=False, sep=';')
df_russia.to_csv('csv/1_task_russia.csv', index=False, sep=';')
display(df.head())
display(df_russia.head())


,Address,Business name,Name,url
0,Ľublanská 2 Bratislava 831 02,Poľnohospodárske družstvo Vajnory,Ing. Branislav Brúder,http://www.orsr.sk/vypis.asp?lan=en&ID=1&SID=2&P=0
1,Pristruhe 44 Bratislava 831 07,Poľnohospodárske družstvo Vajnory,Renáta Sláviková,http://www.orsr.sk/vypis.asp?lan=en&ID=1&SID=2&P=0
2,Dorastenecká 4 Bratislava 831 07,Poľnohospodárske družstvo Vajnory,Peter Ondris,http://www.orsr.sk/vypis.asp?lan=en&ID=1&SID=2&P=0
3,Olšová 6 Bratislava 831 06,Villa Vino Rača a.s.,Ing. Ján Krampl,http://www.orsr.sk/vypis.asp?lan=en&ID=2&SID=2&P=0
4,Gen.Svobodu 12 Pezinok 902 01,Villa Vino Rača a.s.,Ing. Klára Wittlingerová,http://www.orsr.sk/vypis.asp?lan=en&ID=2&SID=2&P=0


,Business name,Name,Address
970,"P. and C. Car, spoločnosť s r.o. v likvidácii P. and C. Car Ltd (anglicky) v likvidácii P. and C. Car G.m.b.H. (nemecky) v likvidácii",Aleksander I. Beskrovnyj,"Rublevskoe cesta 42, byt 91 Moskva Rusko"
971,"P. and C. Car, spoločnosť s r.o. v likvidácii P. and C. Car Ltd (anglicky) v likvidácii P. and C. Car G.m.b.H. (nemecky) v likvidácii",Galina Tchistiakova,Konakovsky 8/1/191 Moskva Rusko
972,"P. and C. Car, spoločnosť s r.o. v likvidácii P. and C. Car Ltd (anglicky) v likvidácii P. and C. Car G.m.b.H. (nemecky) v likvidácii",Oleg V. Protsenko,Lubertski 40-67 Moskva Rusko
973,"P. and C. Car, spoločnosť s r.o. v likvidácii P. and C. Car Ltd (anglicky) v likvidácii P. and C. Car G.m.b.H. (nemecky) v likvidácii",Elena Gribanova,Vesolaja Balka 44/12 Volgograd Rusko
1548,"AVIKO, spol. s r.o.",Igor Volkov,Ak. Vargi 2 Moskva Rusko residence in the Slovak Republic : Záhrebská 8 Bratislava


Функция для сохранения нужных полей страницы

In [83]:
def get_info_page(url):
    page = requests.get(url)
    if page.status_code != 200:
        sleep(5)
        print(url, 'error')
        page = requests.get(url)
    # На сайте используется не utf-8
    page_html = BeautifulSoup(page.content.decode('windows-1250'))
    # Создаем словарь по элементу tl со значениями в тегах ra
    all_tags = [code for code in page_html.find_all('tr') if code != '']
    page_info = {
        [info.get_text().strip()[:-1] for info in tag.find_all('span', attrs={"class": 'tl'})][0]:
        [info.get_text().strip() for info in tag.find_all('span', attrs={"class": 'ra'})]
        for tag in all_tags if [info.get_text().strip()[:-1] for info in tag.find_all('span', attrs={"class": 'tl'})]
    }
    page_info['url'] = url
    return page_info


Парсим страницы, на всякий случай сохраняем дампы словарей, уходим в сон, если закрывают соединение

In [84]:
def get_pages():
    NUM_PAGES = 300000
    start = time.time()
    results = []

    for idx in range(200000, NUM_PAGES):
        sid = 2
        current_url = URL + '?lan=en&ID={}&SID={}&P=0'.format(idx, sid)
        if idx % 100 == 0:
            print(time.time() - start, current_url)
        if idx % 5000 == 0:
            with open('dump/idx{}sid{}.pickle'.format(idx, sid), 'wb') as handle:
                pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)
        try:
            results.append(get_info_page(current_url))
        except Exception:
            print('{} {} fail'.format(idx, current_url))
            sleep(15)
    return results


Обрабатываем результаты

In [85]:
def filter_results(results):
    filtered_results = []
    temp_results = []
    for result in results:
        # Если нет на странице имени компании, пропускаем
        if 'Business name' not in result.keys() or not result['Business name']:
            continue
        # Проверяем, что есть хоть одно поле с нужной информацией присутствует
        columns = set(['Partners', 'Contribution of each member', 'Management body']).intersection(result.keys())
        if not columns:
            continue
        # Заполняем нужные поля в дикте для последующей обработки
        new_info = {'Business name': result['Business name'][0]}
        new_info['url'] = result['url']
        for column in columns:
            new_info[column] = result[column]
        temp_results.append(new_info)
    
    # Для каждой компании вытаскиваем информацию о людях в полях
    for result in temp_results:
        persons_info = {}
        if 'Management body' in result.keys():
            persons_info = process_row(result, persons_info, result['Management body'][2:], 'Management body')
        if 'Partners' in result.keys():
            persons_info = process_row(result, persons_info, result['Partners'], 'Partners')
        if 'Contribution of each member' in result.keys():
            persons_info = process_row(
                result, persons_info, result['Contribution of each member'], 'Contribution of each member'
            )
        if persons_info:
            filtered_results.append(persons_info)

    return filtered_results


Функции необходимые для обработки страниц html к нужному формату

In [140]:
def process_person(result, person_items, persons_info, method):
    COMPANY_SYMBOLS = ['sro', 'inc', 'mbh', 'spol', 'ooo', 'gmbh', 'establishment', 'limited']
    person_info = {}
    name_split_id = 2
    address_split_id = name_split_id
    possible_name = " ".join(person_items[:name_split_id + 1]).lower().replace(' ', '').replace('.', '')
    
    # Удаляем компании из списка, оставляем только людей (Убираем строки с числами, элементами типа gmbh или s.r.o.)
    if 'partner' in possible_name:
        person_items = person_items[1:]
        possible_name = " ".join(person_items[:name_split_id + 1]).lower().replace(' ', '').replace('.', '')
    if (any(c.isdigit() for c in possible_name) or any(symbols in possible_name for symbols in COMPANY_SYMBOLS)):
        return persons_info
    # Определяем где кончается имя человека и начинается его адрес
    if '.' in person_items[0]:
        name_split_id += 1
    address_split_id = name_split_id
    for idx, item in enumerate(person_items):
        if item.startswith('-'):
            address_split_id = idx + 1
            break
    # Записываем результаты в дикт
    person_info['Name'] = " ".join(person_items[:name_split_id])
    if method != 'Contribution of each member':
        person_info['Address'] = " ".join(person_items[address_split_id:])

    # Проверяем, что человека с таким же именем не было в других полях 'Management body' и др.
    if person_info['Name'] not in persons_info.keys():
        persons_info[person_info['Name']] = {'Business name': result['Business name'], 'url': result['url']}
        if method != 'Contribution of each member':
            persons_info[person_info['Name']]['Address'] = person_info['Address']
    return persons_info


# Обрабатываем каждую строку. Добавляем для каждой компании людей из полей 'Management body' или др. с их адресами
def process_row(result, persons_info, row, method):
    person_items = []
    for item in row:
        # Записи имен на сайте разделены датами, будет нашим сплитером
        if re.search(r'\d{2}/\d{2}/\d{4}', item) is not None and person_items:
            persons_info = process_person(result, person_items, persons_info, method)
            person_items = []
        if not re.search(r'\d{2}/\d{2}/\d{4}', item):
            person_items.append(item)
    return persons_info


In [402]:
df = pd.DataFrame(results)

## 3 task

In [210]:
import csv
import pandas as pd
from IPython.display import display, HTML

pd.set_option('display.max_columns', 500)

# Колонки, в которых указаны имена владельцев собственности
PROPRIETORS_COLUMNS = ['Proprietor Name (1)', 'Proprietor Name (2)', 'Proprietor Name (3)', 'Proprietor Name (4)']

# Загружаем все необходимые файлы
ocod_df = pd.read_csv('src/OCOD_FULL_2019_06.csv')
pp_entities_df = pd.read_csv('src/panama_papers.nodes.entity.csv')
pp_officer_df = pd.read_csv('src/panama_papers.nodes.officer.csv')
pp_edges_df = pd.read_csv('src/panama_papers.edges.csv')
pp_edges_df = pp_edges_df.loc[(pp_edges_df['TYPE'] == 'officer_of')]

# Джойним всех иностранных владельцев имущества из Великобритании с их профайлами на PP и соединяем в один DataFrame
super_result = [
    pd.merge(ocod_df, pp_entities_df, left_on=proprietor_column, right_on='name', how='inner').dropna(subset=[proprietor_column])
    for proprietor_column in PROPRIETORS_COLUMNS
]
super_result_df = pd.concat(super_result, ignore_index=True)

# Джойним получившуюся таблицу с владельцами компаний из PP officers (связь компания-владелец из файла PP edges)
super_result_df = pd.merge(super_result_df, pp_edges_df, left_on='node_id', right_on='END_ID', how='left')
super_result_df = pd.merge(super_result_df, pp_officer_df, left_on='START_ID', right_on='node_id', how='left')
super_result_df = super_result_df.rename(columns={
    'Title Number': 'Title_Number', 'Property Address': 'Property_Address',
    'name_x': 'Company_Name', 'name_y': 'Officer'
})[['Title_Number', 'Property_Address', 'Company_Name', 'Officer']]
super_result_df.to_csv('csv/3_task.csv', index=False, sep=';')
display(super_result_df.head())


/Users/nikita2/Desktop/DMIA_Base_2019_Spring/env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (24,28,30,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/nikita2/Desktop/DMIA_Base_2019_Spring/env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/nikita2/Desktop/DMIA_Base_2019_Spring/env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Title_Number,Property_Address,Company_Name,Officer
0,TGL399894,"40 Queensberry House, Friars Lane, Richmond (T...",DEODAR LIMITED,NaN
1,SK355825,"Woodcroft, High Street, Ufford, Woodbridge (IP...",GRAHAM INVESTMENTS LIMITED,NaN
2,SL108367,"4 Slaney Street, Oakengates, Telford (TF2 6ET)",VALFINA DEVELOPMENTS LIMITED,Knocklate Trustees as Trustee of Shoshana Tru
3,SL108367,"4 Slaney Street, Oakengates, Telford (TF2 6ET)",VALFINA DEVELOPMENTS LIMITED,STM FIDUCIAIRE TRUSTEES LIMITED (FORMELY ST GE...
4,SF534020,"15 Oakwood Place, Newcastle (ST5 7BB)",VALFINA DEVELOPMENTS LIMITED,Knocklate Trustees as Trustee of Shoshana Tru


На выходе файл с собственностью, которая может принадлежать нескольким фирмам, а у этих фирм может быть несколько владельцев (или не быть вовсе). 
Не очень понятно, кто такой владелец из PP, я предположил, что это может быть кто угодно со связью 'officer_of' из файла panama_papers.edges.

In [206]:
print(pp_nodes_edges_df.loc[pp_nodes_edges_df['TYPE'] == 'officer_of']['link'].unique())

['shareholder of' 'director / shareholder of' 'director of'
 'power of attorney of' 'director (rami makhlouf) of'
 'director / shareholder / beneficial owner of' 'owner of'
 'member / shareholder of' 'beneficial owner of'
 'owner, director and shareholder of' 'sole shareholder of'
 'president - director of' 'president and director of'
 'director / beneficial owner of' 'power of attorney / shareholder of'
 'director and shareholder of' 'beneficiary of' 'president of'
 'authorized signatory of' 'secretary of'
 'member of foundation council of' 'signatory of'
 'grantee of a mortgage of' 'sole signatory of'
 'sole signatory / beneficial owner of' 'principal beneficiary of'
 'protector of' 'beneficiary, shareholder and director of' 'connected of'
 'first beneficiary of' 'shareholder (through julex foundation) of']
